In [2]:
import numpy as np
import pandas as pd
import sklearn
import cython
import talib
import bokeh
import time


from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from multiprocessing import Pool

from Functions import DataReady, KijunSen, GetData

In [3]:
#GetData

In [4]:
#data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min01.csv")
data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT/ATOMUSDT5min02.csv")

DataReady(data)

,Open,High,Low,Close,Volume
datetime,,,,,
2024-08-12 03:00:00,4.894,4.923,4.892,4.914,17268.2
2024-08-12 03:05:00,4.914,4.929,4.911,4.929,15099.2
2024-08-12 03:10:00,4.929,4.941,4.929,4.938,11389.4
2024-08-12 03:15:00,4.938,4.951,4.934,4.945,9381.2
2024-08-12 03:20:00,4.945,4.945,4.922,4.934,15880.7
...,...,...,...,...,...
2024-08-31 12:50:00,4.596,4.599,4.595,4.599,3811.1
2024-08-31 12:55:00,4.599,4.600,4.593,4.596,2621.8
2024-08-31 13:00:00,4.596,4.603,4.596,4.602,8283.7


In [5]:
class WR_PF_40000I_Data02_Sharpe_Long(Strategy):
    
    # Add variables to the strategy
    atr_lenght = 5
    times_atr_tp = 10
    times_atr_sl = 12
    upper_ema_lenght = 67
    lower_ema_lenght = 199
    stoch_rsi_lenght = 27
    kijun_sen_period = 19
    kijun_sen_price_range = 4

    def init(self):
        # Indicator code
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_lenght)
        self.ema_upper = self.I(talib.EMA, self.data.Close, self.upper_ema_lenght)
        self.ema_lower = self.I(talib.EMA, self.data.Close, self.lower_ema_lenght)
        self.stoch_rsi = self.I(talib.STOCHRSI, self.data.Close, self.stoch_rsi_lenght)
        self.kijun_sen = self.I(KijunSen, data, self.kijun_sen_period)
        self.kijun_sen_to_price = self.data.Close - self.kijun_sen

        
    def next(self):
        if crossover(self.ema_lower, self.ema_upper) and self.kijun_sen_to_price < self.kijun_sen_price_range / 10:
            self.buy(sl = self.data.Close - (self.atr*self.times_atr_sl / 10), tp = self.data.Close + (self.atr*self.times_atr_tp / 10), size = 2000)
        

def do_backtest(Data):

    bt = Backtest(data, WR_PF_40000I_Data02_Sharpe_Long, cash = 10000, hedging = True)

    stats = bt.run()
    bt.plot()
    return (stats)

def do_optimize(Data):

    bt = Backtest(data, WR_PF_40000I_Data02_Sharpe_Long, cash = 10000, hedging = True)

    stats = bt.optimize(
        random_state = 1,
        max_tries = 200,
        return_heatmap = True,
        method = "skopt",
        upper_ema_lenght = range(20, 100, 1),
        lower_ema_lenght = range(100, 200, 1),
        kijun_sen_period = range(10, 40, 1),
        times_atr_tp = range(10, 30, 1),
        times_atr_sl = range(10, 30, 1),
        atr_lenght = range(5, 30, 1),
        stoch_rsi_lenght = range(5, 40, 1),
        kijun_sen_price_range = range(0, 10, 1),
        maximize = "Sharpe Ratio")
    
    bt.plot()
    return(stats)

In [6]:
do_optimize(data)

/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/porsche/Desktop/Turbo-Bot/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')


(Start                     2024-08-12 03:00:00
 End                       2024-08-31 13:10:00
 Duration                     19 days 10:10:00
 Exposure Time [%]                    1.304736
 Equity Final [$]                  9974.185655
 Equity Peak [$]                  10110.897649
 Return [%]                          -0.258143
 Buy & Hold Return [%]               -6.247456
 Return (Ann.) [%]                   -4.607678
 Volatility (Ann.) [%]                7.132378
 Sharpe Ratio                              0.0
 Sortino Ratio                             0.0
 Calmar Ratio                              0.0
 Max. Drawdown [%]                   -1.352125
 Avg. Drawdown [%]                   -0.362861
 Max. Drawdown Duration        9 days 07:30:00
 Avg. Drawdown Duration        1 days 14:04:00
 # Trades                                   14
 Win Rate [%]                        71.428571
 Best Trade [%]                       0.467599
 Worst Trade [%]                     -1.127214
 Avg. Trade [